# Task 2 - LFSR

### 1. Define a generator that implements an LFSR. Given a polynomial and an initial state, it generates an infinite stream of bits. 

In [53]:
from functools import reduce
from operator import xor

def lfsr_generator(poly, state=None):
    ''' generator docstring'''

    #check inputs

    # define variables storing the internal state
    if state == None: 
        s = [True for i in range(max(poly)+1)]
    else: 
        s = []
        for i in state: 
            if i=="1":
                s.append(True)
            else: 
                s.append(False)
    #s.reverse
    #print("Length of s:" + len(s))

    p = [False for i in range(max(poly)+1)]
    for i in range(len(p)):
        if i in poly:
            p[i] = True
    p.reverse()
    p.pop()
    #print("Length of p:" + len(p))
    #print(p)

    b = []
    while len(b) < 10: 

        b.append(s[0])
        #print("b:", b)
        anded = []
        for i in range(len(p)):
            anded.append(s[i] and p[i])
        fb = reduce(xor, anded)


        s.pop(0)
        s.append(fb)
        #print("new s:", s)

    return b

lfsr_generator([3,1,0], "111")

[True, True, True, False, True, False, False, True, True, True]

### 2. Transform the LFSR generator in an iterator, so that it is possible to access to the internal state as an attribute of the class.

In [ ]:
classLFSR(object):''' class docstring '''

    def __init__(self, poly, state=None):
    ''' constructor docstring '''
    #self.poly = poly 
    self.length = len(poly) 
    #self.state = ... 
    self.output = bool
    self.feedback = bool

    #p istedenfor poly
    self.p = [False for i in range(max(poly)+1)]
    for i in range(len(self.p)):
        if i in poly:
            self.p[i] = True
    self.p.reverse()
    self.p.pop()

    #state
    if state == None: 
        self.s = [True for i in range(max(poly)+1)]
    else: 
        self.s = []
        for i in state: 
            if i=="1":
                self.s.append(True)
            else: 
                self.s.append(False)

    def __iter__(self): 
    return self

    def __next__(self): 
        anded = []
        for i in range(len(p)):
            anded.append(s[i] and p[i])
        fb = reduce(xor, anded)
        self.output = s[0]
        self.s.pop(0)
        self.s.append(fb)
    return self.output

    def run_steps(self, N=1): 
        list_of_bool = []
        for i in range(N):
            list_of_bool.append(__next__)
    return list_of_bool 
    
    def cycle(self, state=None): 
        list_of_boo = []
        run_steps(self, 5)
        return list_of_boo



### 3. Define a function that implements the Berlekamp-Massey algorithm which finds the shortest LFSR that can generate the input bit stream.

In [ ]:
defberlekamp_massey(b): 
''' function docstring '''
# algorithm implementation
return poly


### 4. Transform the function implementing the Berlekamp-Massey algorithm into a class that can be applied in a streamingway.

In [ ]:
classBerlekampMassey():
    
    def__init__(self):
    # do stuff
    self.poly = ... 
    
    def__call__(self, bit):
    # do stuff
    return self.poly